<a href="https://colab.research.google.com/github/sebastian122H/Rasselbande/blob/main/Python_for_Data_Science_Die_Rasselbande.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:


import pandas as pd
import numpy as np


In [18]:
df = pd.read_csv("tmdb_movies_data.csv")
df

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,5562,6.5,2015,1.379999e+08,1.392446e+09
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,6185,7.1,2015,1.379999e+08,3.481613e+08
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,2480,6.3,2015,1.012000e+08,2.716190e+08
3,140607,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,...,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,5292,7.5,2015,1.839999e+08,1.902723e+09
4,168259,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,...,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,2947,7.3,2015,1.747999e+08,1.385749e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10861,21,tt0060371,0.080598,0,0,The Endless Summer,Michael Hynson|Robert August|Lord 'Tally Ho' B...,NaN,Bruce Brown,NaN,...,"The Endless Summer, by Bruce Brown, is one of ...",95,Documentary,Bruce Brown Films,6/15/1966,11,7.4,1966,0.000000e+00,0.000000e+00
10862,20379,tt0060472,0.065543,0,0,Grand Prix,James Garner|Eva Marie Saint|Yves Montand|Tosh...,NaN,John Frankenheimer,Cinerama sweeps YOU into a drama of speed and ...,...,Grand Prix driver Pete Aron is fired by his te...,176,Action|Adventure|Drama,Cherokee Productions|Joel Productions|Douglas ...,12/21/1966,20,5.7,1966,0.000000e+00,0.000000e+00
10863,39768,tt0060161,0.065141,0,0,Beregis Avtomobilya,Innokentiy Smoktunovskiy|Oleg Efremov|Georgi Z...,NaN,Eldar Ryazanov,NaN,...,An insurance agent who moonlights as a carthie...,94,Mystery|Comedy,Mosfilm,1/1/1966,11,6.5,1966,0.000000e+00,0.000000e+00
10864,21449,tt0061177,0.064317,0,0,"What's Up, Tiger Lily?",Tatsuya Mihashi|Akiko Wakabayashi|Mie Hama|Joh...,NaN,Woody Allen,WOODY ALLEN STRIKES BACK!,...,"In comic Woody Allen's film debut, he took the...",80,Action|Comedy,Benedict Pictures Corp.,11/2/1966,22,5.4,1966,0.000000e+00,0.000000e+00


In [19]:
df.columns


Index(['id', 'imdb_id', 'popularity', 'budget', 'revenue', 'original_title',
       'cast', 'homepage', 'director', 'tagline', 'keywords', 'overview',
       'runtime', 'genres', 'production_companies', 'release_date',
       'vote_count', 'vote_average', 'release_year', 'budget_adj',
       'revenue_adj'],
      dtype='object')

In [20]:
df.drop(['imdb_id', 'popularity', 'original_title',
       'cast', 'homepage', 'director', 'tagline', 'keywords', 'overview',
       'runtime', 'genres', 'production_companies', 'release_date',
       'vote_count', 'vote_average'],axis = 1, inplace=True)

In [22]:
df.dtypes

id                int64
budget            int64
revenue           int64
release_year      int64
budget_adj      float64
revenue_adj     float64
dtype: object

In [28]:
df

,id,budget,revenue,release_year,budget_adj,revenue_adj
0,135397,150000000,1513528810,2015,1.379999e+08,1.392446e+09
1,76341,150000000,378436354,2015,1.379999e+08,3.481613e+08
2,262500,110000000,295238201,2015,1.012000e+08,2.716190e+08
3,140607,200000000,2068178225,2015,1.839999e+08,1.902723e+09
4,168259,190000000,1506249360,2015,1.747999e+08,1.385749e+09
...,...,...,...,...,...,...
10861,21,0,0,1966,0.000000e+00,0.000000e+00
10862,20379,0,0,1966,0.000000e+00,0.000000e+00
10863,39768,0,0,1966,0.000000e+00,0.000000e+00
10864,21449,0,0,1966,0.000000e+00,0.000000e+00


In [30]:
df = df.replace(0, np.nan)
df = df.dropna(how='all', axis=0)
df.dropna()

,id,budget,revenue,release_year,budget_adj,revenue_adj
0,135397,150000000.0,1.513529e+09,2015,1.379999e+08,1.392446e+09
1,76341,150000000.0,3.784364e+08,2015,1.379999e+08,3.481613e+08
2,262500,110000000.0,2.952382e+08,2015,1.012000e+08,2.716190e+08
3,140607,200000000.0,2.068178e+09,2015,1.839999e+08,1.902723e+09
4,168259,190000000.0,1.506249e+09,2015,1.747999e+08,1.385749e+09
...,...,...,...,...,...,...
10822,396,7500000.0,3.373669e+07,1966,5.038511e+07,2.266436e+08
10828,5780,3000000.0,1.300000e+07,1966,2.015404e+07,8.733419e+07
10829,6644,4653000.0,6.000000e+06,1966,3.125892e+07,4.030809e+07
10835,5923,12000000.0,2.000000e+07,1966,8.061618e+07,1.343603e+08
